<a href="https://colab.research.google.com/github/lakecg2/practica_progavan/blob/main/biblioteca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import datetime, timedelta
from typing import List

### Clases base y derivadas
class Material:
    def __init__(self, titulo: str, estado: str = "disponible"):
        self.titulo = titulo
        self.estado = estado

    def __str__(self):
        return f"{self.titulo} - {self.estado}"

class Libro(Material):
    def __init__(self, titulo: str, autor: str, genero: str, estado: str = "disponible"):
        super().__init__(titulo, estado)
        self.autor = autor
        self.genero = genero

    def __str__(self):
        return f"Libro: {self.titulo} por {self.autor} ({self.genero}) - {self.estado}"

class Revista(Material):
    def __init__(self, titulo: str, edicion: str, periodicidad: str, estado: str = "disponible"):
        super().__init__(titulo, estado)
        self.edicion = edicion
        self.periodicidad = periodicidad

    def __str__(self):
        return f"Revista: {self.titulo} (Edición {self.edicion}, {self.periodicidad}) - {self.estado}"

class MaterialDigital(Material):
    def __init__(self, titulo: str, tipo_archivo: str, enlace_descarga: str, estado: str = "disponible"):
        super().__init__(titulo, estado)
        self.tipo_archivo = tipo_archivo
        self.enlace_descarga = enlace_descarga

    def __str__(self):
        return f"Material Digital: {self.titulo} ({self.tipo_archivo}) - {self.estado}"

class Persona:
    def __init__(self, nombre: str, identificacion: str):
        self.nombre = nombre
        self.identificacion = identificacion

class Usuario(Persona):
    def __init__(self, nombre: str, identificacion: str):
        super().__init__(nombre, identificacion)
        self.materiales_prestados = []
        self.penalizaciones = []

    def devolver_material(self, material, sucursal):
        if material in self.materiales_prestados:
            self.materiales_prestados.remove(material)
            material.estado = "disponible"
            sucursal.agregar_material(material)
            print(f"{self.nombre} ha devuelto {material.titulo}.")
        else:
            print(f"{self.nombre} no tiene prestado {material.titulo}.")

class Bibliotecario(Persona):
    def __init__(self, nombre: str, identificacion: str):
        super().__init__(nombre, identificacion)

    def agregar_material(self, material, sucursal):
        sucursal.agregar_material(material)
        print(f"{self.nombre} ha agregado {material.titulo} a {sucursal.nombre}.")

    def gestionar_prestamo(self, usuario, material, sucursal):
        if material.estado == "disponible":
            usuario.materiales_prestados.append(material)
            material.estado = "prestado"
            sucursal.remover_material(material)
            prestamo = Prestamo(usuario, material)
            print(f"{self.nombre} ha prestado {material.titulo} a {usuario.nombre}.")
            return prestamo
        else:
            print(f"{material.titulo} no está disponible para préstamo.")
            return None

    def transferir_material(self, material, sucursal_origen, sucursal_destino):
        if material in sucursal_origen.catalogo:
            sucursal_origen.remover_material(material)
            sucursal_destino.agregar_material(material)
            print(f"{self.nombre} ha transferido {material.titulo} de {sucursal_origen.nombre} a {sucursal_destino.nombre}.")
        else:
            print(f"{material.titulo} no está en {sucursal_origen.nombre}.")

class Sucursal:
    def __init__(self, nombre: str):
        self.nombre = nombre
        self.catalogo = []

    def agregar_material(self, material):
        self.catalogo.append(material)

    def remover_material(self, material):
        if material in self.catalogo:
            self.catalogo.remove(material)
        else:
            print(f"{material.titulo} no está en el catálogo de {self.nombre}.")

    def buscar_disponibles(self):
        return [material for material in self.catalogo if material.estado == "disponible"]

class Prestamo:
    def __init__(self, usuario: Usuario, material: Material):
        self.usuario = usuario
        self.material = material
        self.fecha_prestamo = datetime.now()
        self.fecha_devolucion = self.fecha_prestamo + timedelta(days=14)

    def calcular_penalizacion(self):
        if datetime.now() > self.fecha_devolucion:
            dias_retraso = (datetime.now() - self.fecha_devolucion).days
            return dias_retraso * 1  # $1 por día de retraso
        return 0

class Penalizacion:
    def __init__(self, usuario: Usuario, monto: float):
        self.usuario = usuario
        self.monto = monto

    def aplicar_penalizacion(self):
        self.usuario.penalizaciones.append(self)
        print(f"Se ha aplicado una penalización de ${self.monto} a {self.usuario.nombre}.")

class Catalogo:
    def __init__(self, sucursales: List[Sucursal]):
        self.sucursales = sucursales

    def buscar_disponibles(self):
        disponibles = []
        for sucursal in self.sucursales:
            disponibles.extend(sucursal.buscar_disponibles())
        return disponibles

    def buscar_por_autor(self, autor: str):
        return [material for sucursal in self.sucursales for material in sucursal.catalogo if isinstance(material, Libro) and material.autor == autor]

    def buscar_por_genero(self, genero: str):
        return [material for sucursal in self.sucursales for material in sucursal.catalogo if isinstance(material, Libro) and material.genero == genero]

    def buscar_por_tipo(self, tipo: type):
        return [material for sucursal in self.sucursales for material in sucursal.catalogo if isinstance(material, tipo)]

### Menú y programa principal
def mostrar_menu():
    print("\n--- Menú de la Biblioteca ---")
    print("1. Consultar catálogo de materiales disponibles")
    print("2. Reservar un libro")
    print("3. Devolver un libro")
    print("4. Transferir material entre sucursales")
    print("5. Ver penalizaciones de un usuario")
    print("6. Salir")

def main():
    # Crear sucursales y materiales de ejemplo
    sucursal1 = Sucursal("Sucursal Central")
    sucursal2 = Sucursal("Sucursal Norte")

    libro1 = Libro("Cien Años de Soledad", "Gabriel García Márquez", "Realismo Mágico")
    revista1 = Revista("National Geographic", "Enero 2023", "Mensual")
    material_digital1 = MaterialDigital("Python for Beginners", "PDF", "http://example.com/python.pdf")

    sucursal1.agregar_material(libro1)
    sucursal2.agregar_material(revista1)
    sucursal1.agregar_material(material_digital1)

    # Crear usuarios y bibliotecarios de ejemplo
    usuario1 = Usuario("Juan Pérez", "12345")
    bibliotecario1 = Bibliotecario("Ana López", "67890")

    # Crear catálogo global
    catalogo = Catalogo([sucursal1, sucursal2])

    while True:
        mostrar_menu()
        opcion = input("Seleccione una opción: ")

        if opcion == "1":  # Consultar catálogo
            print("\nMateriales disponibles:")
            disponibles = catalogo.buscar_disponibles()
            if disponibles:
                for material in disponibles:
                    print(material)
            else:
                print("No hay materiales disponibles en este momento.")

        elif opcion == "2":  # Reservar un libro
            titulo = input("Ingrese el título del libro que desea reservar: ")
            materiales = catalogo.buscar_disponibles()
            libro_encontrado = None

            for material in materiales:
                if isinstance(material, Libro) and material.titulo.lower() == titulo.lower():
                    libro_encontrado = material
                    break

            if libro_encontrado:
                prestamo = bibliotecario1.gestionar_prestamo(usuario1, libro_encontrado, sucursal1)
                if prestamo:
                    print(f"Libro '{libro_encontrado.titulo}' reservado con éxito.")
            else:
                print(f"No se encontró el libro '{titulo}' disponible.")

        elif opcion == "3":  # Devolver un libro
            titulo = input("Ingrese el título del libro que desea devolver: ")
            materiales_prestados = usuario1.materiales_prestados
            libro_encontrado = None

            for material in materiales_prestados:
                if isinstance(material, Libro) and material.titulo.lower() == titulo.lower():
                    libro_encontrado = material
                    break

            if libro_encontrado:
                usuario1.devolver_material(libro_encontrado, sucursal1)
                # Calcular penalización si hay retraso
                for prestamo in usuario1.materiales_prestados:
                    if prestamo.material == libro_encontrado:
                        penalizacion = Penalizacion(usuario1, prestamo.calcular_penalizacion())
                        if penalizacion.monto > 0:
                            penalizacion.aplicar_penalizacion()
                        break
            else:
                print(f"No tiene prestado el libro '{titulo}'.")

        elif opcion == "4":  # Transferir material entre sucursales
            titulo = input("Ingrese el título del material que desea transferir: ")
            materiales = catalogo.buscar_disponibles()
            material_encontrado = None

            for material in materiales:
                if material.titulo.lower() == titulo.lower():
                    material_encontrado = material
                    break

            if material_encontrado:
                print("Sucursales disponibles:")
                print(f"1. {sucursal1.nombre}")
                print(f"2. {sucursal2.nombre}")
                sucursal_destino_num = input("Seleccione la sucursal de destino (1 o 2): ")

                if sucursal_destino_num == "1":
                    bibliotecario1.transferir_material(material_encontrado, sucursal2, sucursal1)
                elif sucursal_destino_num == "2":
                    bibliotecario1.transferir_material(material_encontrado, sucursal1, sucursal2)
                else:
                    print("Opción no válida.")
            else:
                print(f"No se encontró el material '{titulo}' disponible.")

        elif opcion == "5":  # Ver penalizaciones de un usuario
            if usuario1.penalizaciones:
                print(f"Penalizaciones de {usuario1.nombre}:")
                for penalizacion in usuario1.penalizaciones:
                    print(f"- ${penalizacion.monto}")
            else:
                print(f"{usuario1.nombre} no tiene penalizaciones.")

        elif opcion == "6":  # Salir
            print("Gracias por usar el sistema de la biblioteca. ¡Hasta luego!")
            break

        else:
            print("Opción no válida. Intente nuevamente.")

# Ejecutar el programa
if __name__ == "__main__":
    main()